<a href="https://colab.research.google.com/github/robicastilho/remedio_certo/blob/main/remedio_certo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#Instale a SDK Google Gen AI
!pip -q install google-genai
!pip install -q google-adk
!pip install -q python-magic

#Imports e configuração do Colab
from google import genai
from google.genai import types
from google.colab import files
from google.colab import userdata
import os

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import magic # Para detectar o MIME type (instale com: pip install python-magic)
from typing import List

warnings.filterwarnings("ignore")

###########################################################
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
###########################################################
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

###########################################################
# Função auxiliar similar a anterior, mas que envia também o caminho de um arquivo (Fornecida pelo Gemini)
###########################################################
def call_agent_multimodal(agent: Agent, message_text: str, image_file_path: str = None) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1") # Adapte IDs se necessário
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    list_of_parts = [types.Part(text=message_text)]

    if image_file_path:
        try:
            # Detectar o MIME type
            mime_type = magic.from_file(image_file_path, mime=True)
            with open(image_file_path, "rb") as f:
                image_data = f.read()
            list_of_parts.append(types.Part(inline_data=types.Blob(mime_type=mime_type, data=image_data)))
        except Exception as e:
            print(f"Erro ao processar a imagem {image_file_path}: {e}")
            # Decide como lidar com o erro, talvez levantar uma exceção ou prosseguir sem a imagem

    content = types.Content(role="user", parts=list_of_parts)

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content): # Adapte IDs se necessário
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response.strip()

###################################################
# --- Agente: Farmacêutico Decifrador de Receitas --- #
###################################################
def agente_farmaceutico(file_path: str, especialidade_medica: str, sintomas_paciente: str):
    """
    Cria e executa um agente farmacêutico para analisar uma imagem de receita médica.

    Args:
        file_path (str): O caminho para o arquivo de imagem da receita médica.
                         IMPORTANTE: A função call_agent precisará ser adaptada
                         para carregar e enviar este arquivo como um Part de imagem.
        especialidade_medica (str): A especialidade do médico prescritor.
        sintomas_paciente (str): Os sintomas relatados pelo paciente.

    Returns:
        str: A análise da receita feita pelo agente.
    """
    farmaceutico = Agent(
        name="agente_farmaceutico",
        model=model_name,

        instruction=f"""
          Você é um Farmacêutico altamente experiente e especializado em decifrar receitas médicas manuscritas, com profundo conhecimento em farmacologia, caligrafia médica e abreviações comuns.

          **Você já tem a informação de que a especialidade médica do prescritor é {especialidade_do_prescritor} e os sintomas do paciente {sintomas_paciente}.**

          Sua tarefa principal é analisar uma imagem de uma receita médica manuscrita e extrair as informações cruciais sobre os medicamentos prescritos.

          **Para cada item identificado na receita, forneça as seguintes informações em um formato claro e objetivo:**

          * **Medicamento:** Nome do medicamento (se houver incerteza, inclua possíveis alternativas e uma observação).
          * **Posologia:** Dose, via de administração e frequência (ex: "500mg, via oral, a cada 8 horas").
          * **Duração do Tratamento:** Período pelo qual o medicamento deve ser administrado (ex: "por 7 dias").
          * **Grau de Confiança:** (ALTO, MÉDIO, BAIXO) na sua identificação, para cada campo (Medicamento, Posologia, Duração).
          * **Observações/Incertezas:** Detalhes sobre qualquer dificuldade na leitura, caracteres ilegíveis ou suposições feitas.

          ---

          **Passos para a identificação (em ordem de prioridade):**

          1.  **Decifração Direta:** Comece lendo e transcrevendo o que o médico escreveu com a maior precisão possível.
          2.  **Análise de Contexto e Caligrafia:** Utilize seu conhecimento em farmacologia, abreviações comuns em receitas e, crucialmente, a **informação da especialidade médica e sintomas do paciente** para inferir palavras ou partes ilegíveis.
          3.  **Busca por Similares (Grafia):** Se a palavra for incerta ou o medicamento encontrado não seja para tratamento dos sintomas informados, pesquise por nomes de medicamentos ou termos de posologia com grafia similar ou fonética próxima que se encaixem no contexto.
          4.  **Identificação de Ilegíveis:** Se, após os passos anteriores, uma parte da receita permanecer impossível de decifrar, classifique-a como "Não identificado" ou "Ilegível" e atribua um grau de confiança "BAIXO". **Nunca invente informações ou nomes de medicamentos.**
          5.  **Validação e Refinamento Contextual:**
              * Com base no nome do medicamento que você identificou (mesmo que com confiança baixa), **pesquise no (google_search) por medicamentos que tenham nomes iguais ou muito similares**.
              * Simultaneamente, avalie se os medicamentos encontrados nessa pesquisa e o medicamento originalmente identificado **possuem ação terapêutica compatível com os sintomas informados pelo paciente e são comumente prescritos por médicos com a mesma especialidade** para essa condição.
              * Se esta análise contextual sugerir fortemente um medicamento diferente do inicialmente decifrado (mas graficamente similar e contextualmente mais plausível), **corrija a informação do medicamento** e ajuste o Grau de Confiança para cima, justificando a alteração em "Observações/Incertezas". Caso a compatibilidade seja baixa, mas a legibilidade da receita persista, mantenha a incerteza e o Grau de Confiança original ou até o diminua.
          ---
        """,
        description="Agente farmacêutico que analisa receitas médicas manuscritas e usa o Google Search para validação.",
        tools=[google_search] # Garanta que 'Google Search' esteja definido como uma Tool
    )

    # Esta entrada de texto contextualiza a imagem que DEVE ser enviada à parte.
    entrada_do_agente_farmaceutico = (
        f"Especialidade médica do prescritor: {especialidade_medica}\n"
        f"Sintomas do paciente: {sintomas_paciente}\n\n"
        "Por favor, analise a IMAGEM da receita médica que acompanha esta solicitação e siga as instruções detalhadas."
    )

    # IMPORTANTE: A função call_agent precisará ser modificada para aceitar
    # o file_path, carregar a imagem e passá-la como um 'Part' de imagem
    # junto com o 'Part' de texto.
    # Exemplo conceitual de como seria a chamada (a definir em call_agent):
    # analise_da_receita = call_agent(farmaceutico, entrada_do_agente_farmaceutico, image_file_path=file_path)

    # Por enquanto, chamando como nos exemplos anteriores, mas ciente da limitação:
    # Esta chamada NÃO envia a imagem com a função call_agent atual.
    analise_da_receita = call_agent_multimodal(farmaceutico, entrada_do_agente_farmaceutico, image_file_path=file_path)

    return analise_da_receita

##############################################################
# --- Agente: Identificador de Especialidade Médica --- #
##############################################################
def agente_identificador_especialidade(texto_para_analise: str) -> str:
    """
    Cria e executa um agente para identificar uma especialidade médica a partir de um texto.

    Args:
        texto_para_analise (str): O texto que pode conter uma menção a uma especialidade médica.

    Returns:
        str: A especialidade médica identificada (ex: "Cardiologista", "Dermatologista")
             ou "clinico geral" se nenhuma especialidade for claramente identificada.
    """

    # Instrução detalhada para o agente
    instrucao_para_agente = """
    Você é um assistente inteligente especializado em terminologia e reconhecimento de especialidades médicas.
    Sua principal tarefa é analisar o texto fornecido pelo usuário e identificar se ele se refere a uma especialidade médica conhecida.

    Siga estas diretrizes rigorosamente:

    1.  **Análise do Texto:** Examine o texto de entrada cuidadosamente. O texto pode conter o nome de uma especialidade (ex: "Oftalmologia"), o nome de um médico especialista (ex: "urologista") ou uma descrição da área (ex: "especialista em doenças de pele").

    2.  **Identificação da Especialidade:**
        * Se você identificar claramente uma especialidade médica, retorne o nome padronizado dessa especialidade.
            Exemplos de mapeamento para o retorno esperado:
            - Se a entrada for "cardiologia" ou "médico do coração" ou "cardiologista", retorne "Cardiologista".
            - Se a entrada for "dermatologia" ou "especialista de pele" ou "dermatologista", retorne "Dermatologista".
            - Se a entrada for "urologia" ou "urologista", retorne "Urologista".
            - Se a entrada for "clínica geral", "médico de família" ou "clinico geral", retorne "clinico geral".
            - Se a entrada for "pediatria" ou "médico de crianças", retorne "Pediatra".
        * Seja flexível com variações comuns, mas mantenha a precisão.

    3.  **Retorno Padrão (Fallback):**
        * Se o texto fornecido NÃO contiver uma referência clara a uma especialidade médica reconhecível,
        * OU se for ambíguo,
        * OU se for um texto genérico que não se relaciona com especialidades médicas,
        * ENTÃO, você DEVE retornar a string "clinico geral".

    4.  **Formato da Saída:**
        * Sua resposta deve ser ÚNICA E EXCLUSIVAMENTE o nome da especialidade identificada (conforme item 2) ou a string "clinico geral" (conforme item 3).
        * Não inclua NENHUMA palavra, frase, explicação, pontuação ou formatação adicional. Apenas a string resultante.
          Por exemplo, se identificar "Urologista", sua saída deve ser exatamente "Urologista". Se o fallback for ativado, sua saída deve ser exatamente "clinico geral".
    """

    # Criação da instância do agente
    identificador_agente = Agent(
        name="agente_identificador_especialidade",
        model=model_name,
        instruction=instrucao_para_agente,
        description="Agente especialista em identificar especialidades médicas em texto ou retornar 'clinico geral'.",
        # Nenhuma ferramenta externa (como Google Search) é necessária para esta tarefa.
        # tools=[] # Opcional, pode ser omitido se não houver ferramentas
    )

    # Preparação da entrada para o agente
    # A instrução é genérica, então passamos o texto específico para análise aqui.
    entrada_para_agente = f"Por favor, analise o seguinte texto para identificar a especialidade médica: \"{texto_para_analise}\""

    # Execução do agente
    especialidade_identificada = call_agent(identificador_agente, entrada_para_agente)

    # A função 'call_agent' pode retornar strings com novas linhas no final, dependendo da sua implementação.
    # O .strip() garante que apenas o texto principal seja retornado, removendo espaços em branco
    # ou novas linhas no início ou fim, assegurando o formato "unicamente a especialidade".
    return especialidade_identificada.strip()

###################################################
# --- Agente 7: Identificador de Sintomas --- #
###################################################
def agente_identificador_sintomas(sintomas_paciente: str) -> str:
    """
    Cria e executa um agente para identificar sintomas de um paciente a partir de um texto.

    Args:
        sintomas_paciente (str): O texto descrevendo os possíveis sintomas do paciente.

    Returns:
        str: Os sintomas identificados (ex: "dor de cabeça", "febre e tosse")
             ou "são desconhecidos" se nenhum sintoma for claramente identificado.
    """

    # Instrução detalhada para o agente
    instrucao_para_agente = """
    Você é um assistente médico altamente competente, treinado para identificar e extrair sintomas ou doenças de pacientes
    a partir de descrições textuais. Sua principal tarefa é analisar o texto fornecido pelo usuário
    e determinar de forma concisa quais são os sintomas ou doenças relatados.

    Siga estas diretrizes rigorosamente:

    1.  **Análise do Texto:** Examine o texto de entrada fornecido pelo usuário com atenção. O texto pode conter
        descrições de dores, desconfortos, alterações no estado de saúde ou outras condições que
        configuram sintomas.

    2.  **Identificação e Extração de Sintomas ou Doenças:**
        * Se o texto descrever um ou mais sintomas ou doenças de forma clara (por exemplo, "Estou com uma dor de cabeça terrível",
            "Sinto febre e calafrios constantes", "Tenho tido muita tosse seca e uma leve dor de garganta", "Estou com covid"),
            seu objetivo é extrair a essência desses sintomas ou doenças.
        * Seja conciso e direto na descrição do sintoma ou doença retornado. Por exemplo:
            - Se a entrada for "Minha cabeça está latejando muito", um bom retorno seria "dor de cabeça".
            - Se a entrada for "Estou com dor de cabeça e também um pouco de náusea", o retorno deve ser "dor de cabeça e náusea".
            - Se a entrada for "Tenho sentido dores pelo corpo todo e um cansaço extremo", retorne "dores pelo corpo e cansaço extremo".
            - Se a entrada for "Estou com covid 19", retorne "covid".


    3.  **Formato do Retorno (Sintomas ou doenças Encontrados):**
        * Se um único sintoma ou doença for identificado, retorne APENAS a descrição desse sintoma ou doença.
            Exemplo de entrada: "Muita coriza." -> Retorno esperado: "coriza".
        * Se múltiplos sintomas ou doenças forem identificados, liste-os em uma única string, de forma natural, geralmente separados por " e ".
            Se houver mais de dois, pode usar vírgulas e " e " antes do último.
            Exemplo de entrada: "Tosse, febre alta e dor muscular." -> Retorno esperado: "tosse, febre alta e dor muscular".
            Exemplo de entrada: "Azia e queimação no estômago." -> Retorno esperado: "azia e queimação no estômago".
        * O objetivo é que o retorno seja uma string única representando os sintomas ou doenças de forma clara.

    4.  **Retorno Padrão (Sintomas ou doenças Não Identificados/Desconhecidos):**
        * Se o texto fornecido NÃO descrever nenhum sintoma ou doença médico reconhecível (por exemplo, "Estou me sentindo bem hoje!",
            "Gostaria de agendar um retorno", "Qual o horário de funcionamento?"),
        * OU se a descrição for excessivamente vaga, subjetiva demais e não permitir a identificação de um sintoma ou doença específico
            (por exemplo, "Não me sinto eu mesmo", "algo está estranho"),
        * ENTÃO, você DEVE retornar a string "são desconhecidos".

    5.  **Exclusividade da Saída:**
        * Sua resposta deve ser ÚNICA E EXCLUSIVAMENTE a string contendo os sintomas ou doenças identificados (conforme item 3)
            ou a string "são desconhecidos" (conforme item 4).
        * Não inclua NENHUMA palavra, frase introdutória (como "Os sintomas são:"), explicação,
            pontuação desnecessária ou formatação extra. Apenas a string resultante.
    """

    # Criação da instância do agente
    identificador_sintomas_agente = Agent(
        name="agente_identificador_sintomas",
        model=model_name,  # Modelo eficiente para tarefas de NLU
        instruction=instrucao_para_agente,
        description="Agente especialista em identificar sintomas de pacientes em texto ou retornar 'são desconhecidos'.",
        # Nenhuma ferramenta externa é necessária.
    )

    # Preparação da entrada para o agente
    entrada_para_agente = f"Por favor, analise a descrição a seguir e identifique os sintomas do paciente: \"{sintomas_paciente}\""

    # Execução do agente
    sintomas_identificados = call_agent(identificador_sintomas_agente, entrada_para_agente)

    # Remover espaços/linhas em branco extras para garantir a saída limpa
    return sintomas_identificados.strip()

#################################################################
# --- Agente: Buscador de Medicamentos Online --- #
#################################################################
def agente_buscador_medicamentos_online(receita_analisada: str, data_de_hoje) -> str:
    """
    Cria e executa um agente para buscar opções de compra online para medicamentos
    listados em um texto de receita.

    Args:
        receita_analisada (str): Um texto contendo os nomes dos medicamentos
                                (e opcionalmente dosagens/formas) de uma receita.

    Returns:
        str: Um relatório formatado com até 3 opções de compra online para cada
             medicamento, incluindo Nome do Medicamento, Preço, Local e URL,
             ou uma mensagem indicando que nenhum medicamento/opção foi encontrado.
    """

    # Instrução detalhada para o agente
    instrucao_para_agente = f"""
    Você é um assistente de pesquisa avançado, especialista em encontrar informações sobre a compra de medicamentos
    online no Brasil. Sua data de conhecimento atual é {data_de_hoje}.
    Sua tarefa é analisar o texto de uma receita fornecida, identificar cada medicamento e, para cada um,
    usar a ferramenta de busca do Google (google_search) para encontrar até três opções de compra online.

    Texto da Receita Analisada Fornecido pelo Usuário:
    "{receita_analisada}"

    Siga estas diretrizes rigorosamente:

    1.  **Identificação dos Medicamentos:**
        * Analise o texto da receita fornecido acima.
        * Identifique cada nome de medicamento. O texto pode incluir dosagens (ex: "Losartana 50mg") ou
            formas farmacêuticas (ex: "Paracetamol gotas"). Use o nome completo, incluindo dosagem/forma se disponível,
            para a sua pesquisa e relatório, pois isso afeta o preço e a disponibilidade.

    2.  **Pesquisa Online com a ferramenta de busca do Google (google_search) (para CADA medicamento):**
        * Para cada medicamento identificado, execute uma pesquisa usando a ferramenta de busca do Google (google_search).
            Formule suas buscas de forma eficaz (ex: "comprar [Nome do Medicamento com Dosagem] online Brasil preço").
        * Procure por farmácias online ou drogarias conhecidas que operem no Brasil.
        * O objetivo é encontrar:
            a.  **Nome do Local/Site:** O nome da farmácia online (ex: "Droga Raia", "Pague Menos", "Drogaria São Paulo").
            b.  **Preço:** O preço atual do medicamento no site. Se o preço variar muito (ex: diferentes quantidades de comprimidos),
                tente encontrar o mais comum ou o que corresponda à dosagem/forma, se especificada. Se o preço não estiver
                claramente visível ou acessível nos primeiros resultados, indique "Preço não encontrado".
            c.  **URL:** O link direto para a página do produto ou uma página de resultados de busca dentro do site da farmácia
                onde o medicamento pode ser encontrado. Nunca retorne URLs do dominio 'vertexaisearch.cloud.google.com'. Sempre retorne a URL final da página do medicamento.


     3.  **Para cada medicamento pesquisado, liste até 3 (três) opções de compra encontradas em um formato claro e objetivo, seguindo o formato abaixo:**

          * Opção 1:
          * **Medicamento:** [Nome do Medicamento Conforme Identificado/Pesquisado]
          * **Local:** [Nome do Site/Farmácia]
          * **Preço:** [Preço Encontrado ou "Preço não encontrado"]
          * **URL:** [https://www.youtube.com/shorts/8XDxvI1d4Bc](https://www.youtube.com/shorts/8XDxvI1d4Bc)

          * Opção 2:
          * **Medicamento:** [Nome do Medicamento Conforme Identificado/Pesquisado]
          * **Local:** [Nome do Site/Farmácia]
          * **Preço:** [Preço Encontrado ou "Preço não encontrado"]
          * **URL:** [https://www.youtube.com/shorts/8XDxvI1d4Bc](https://www.youtube.com/shorts/8XDxvI1d4Bc)

          * Opção 3:
          * **Medicamento:** [Nome do Medicamento Conforme Identificado/Pesquisado]
          * **Local:** [Nome do Site/Farmácia]
          * **Preço:** [Preço Encontrado ou "Preço não encontrado"]
          * **URL:** [https://www.youtube.com/shorts/8XDxvI1d4Bc](https://www.youtube.com/shorts/8XDxvI1d4Bc)
            ---
            ```
        * Se houver mais de um medicamento na receita, repita a estrutura acima para cada um, separados por "---".
        * Se, após uma busca diligente com a ferramenta de busca do Google (google_search, nenhuma opção de compra online for encontrada para um
            medicamento específico, indique da seguinte forma:
            ```text
            Medicamento: [Nome do Medicamento Conforme Identificado/Pesquisado]
            Nenhuma opção de compra online encontrada através da busca.
            ---
            ```
        * Se o texto da `receita_analisada` fornecido inicialmente não contiver nenhum nome de medicamento
            que você possa identificar, sua resposta deve ser APENAS:
            "Nenhum medicamento identificável encontrado na receita analisada."

    4.  **Considerações Adicionais:**
        * Sempre utilize a ferramenta de busca do Google (google_search) para cada medicamento. Não use conhecimento prévio sobre preços ou URLs,
            pois eles mudam constantemente.
        * Seja objetivo e forneça apenas as informações solicitadas no formato especificado.
        * Certifique-se de que as URLs sejam válidas e apontem para sites brasileiros.
    """

    # Criação da instância do agente
    # É crucial que 'Google Search' aqui seja a Tool configurada para o Gemini.
    buscador_medicamentos_agente = Agent(
        name="agente_buscador_medicamentos_online",
        model=model_name,
        instruction=instrucao_para_agente,
        description="Agente que busca informações de compra online para medicamentos usando o Google Search.",
        tools=[google_search] # Passando a ferramenta Google Search para o agente
    )

    # Preparação da entrada para o agente.
    # A instrução já inclui o 'receita_analisada' através do f-string,
    # mas é boa prática também ter um prompt de ativação claro.
    # Neste caso, como a instrução é montada dinamicamente com a receita,
    # uma mensagem simples de ativação é suficiente.
    entrada_para_agente = "Por favor, processe a receita fornecida em suas instruções e encontre as opções de compra."

    # Execução do agente
    relatorio_compra = call_agent(buscador_medicamentos_agente, entrada_para_agente)

    # Remover espaços/linhas em branco extras para garantir a saída limpa
    return relatorio_compra.strip()

#################################################################
# --- MAIN COMECA AQUI --- #
#################################################################
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
#model_name = "gemini-2.0-flash"
model_name = "gemini-2.5-flash-preview-04-17"

# Mensagem de boas-vindas e explicação do projeto
# A explicação (após o título e emoji de boas-vindas) tem aproximadamente 162 caracteres.
mensagem_markdown = """
# Bem-vindo(a) ao **Remédio Certo**! 👋

Projeto do Robinson Castilho, nascido de uma experiência pessoal.
Minha missão: decifrar receitas médicas 📜 ilegíveis com IA 🤖 para sua segurança e saúde! 😊

Para começar por favor faça o upload de uma imagem de uma receita médica.


"""

# Exibe a mensagem formatada como Markdown
display(Markdown(mensagem_markdown))

#Widget de upload
uploaded = files.upload()
image_path = next(iter(uploaded.keys()))

# Exibe um título para a seção, mantendo a consistência se for parte de um formulário
display(Markdown("### 👨‍⚕️ **Sobre o(a) Prescritor(a)**")) # Título inclusivo
display(Markdown("Agora, algumas informações sobre quem prescreveu a receita:"))
display(Markdown("**Qual a especialidade do(a) Médico(a)?** 🩺"))

# O input agora pode ter um prompt mais direto e com exemplos
especialidade_do_prescritor = input("➡️ Digite a especialidade aqui (ex: Cardiologia, Pediatria, Clínica Geral): ")


# Exibe uma introdução/pergunta formatada em Markdown
display(Markdown("### 🩺 **Informações do Paciente**")) # Um título para a seção
display(Markdown("Para continuarmos, preciso de alguns detalhes:"))
display(Markdown("**E quais sintomas ou doenças o paciente possuía?** 🤔"))

# O input agora pode ter um prompt mais simples, já que a pergunta foi feita acima
sintomas_paciente = input("➡️ Digite os sintomas/doenças aqui (ex: febre, tosse persistente, diabetes): ")


#Agente que valida o texto informado pelo usuario afim de encontrar um nome valido para a especialidade médica do prescritor
especialidade_do_prescritor = agente_identificador_especialidade(especialidade_do_prescritor)
#print(f"Especialidade Encontrada ou inferida: {especialidade_do_prescritor}")

#Agente que valida o texto informado pelo usuario afim de encontrar um nome valido para os sintimas ou doenças do paciente
sintomas_paciente = agente_identificador_sintomas(sintomas_paciente)
#print(f"Sintomas/Doenças Encontrados ou inferidos: {sintomas_paciente}")

#Agente que 'traduz' os garanchos em algo humanamente entendível
receita_analisada = agente_farmaceutico(image_path, especialidade_do_prescritor, sintomas_paciente)
display(Markdown((receita_analisada)))

display(Markdown(("Agora vou procurar por opções para você adquirir os medicamento. Aguarde um instante")))

#Agente que busca os preços dos medicamentos para auxiliar o usuário a entender qual será o custo do tratamento
buscador_precos = agente_buscador_medicamentos_online(receita_analisada, date.today().strftime("%d/%m/%Y"))
display(Markdown((buscador_precos)))


# Mensagem de pedido de voto
# O corpo do pedido (após o título) tem aproximadamente 198 caracteres.
pedido_voto_markdown = """
### Apoie o Remédio Certo! 🗳️✨

Gostou do **Remédio Certo**? 💊 Ajude essa ideia a decolar! 🚀
Vote no projeto na **Imersão AI Alura + Google**, lá no Discord.
Seu voto faz toda a diferença! 🙏 #RemedioCerto #ImersaoIA
"""

# Exibe a mensagem formatada como Markdown
display(Markdown(pedido_voto_markdown))


# Bem-vindo(a) ao **Remédio Certo**! 👋

Projeto do Robinson Castilho, nascido de uma experiência pessoal.
Minha missão: decifrar receitas médicas 📜 ilegíveis com IA 🤖 para sua segurança e saúde! 😊

Para começar por favor faça o upload de uma imagem de uma receita médica.




Saving receita-medica-exemplo-covid-clinico-geral.jpg to receita-medica-exemplo-covid-clinico-geral (6).jpg


### 👨‍⚕️ **Sobre o(a) Prescritor(a)**

Agora, algumas informações sobre quem prescreveu a receita:

**Qual a especialidade do(a) Médico(a)?** 🩺

...

➡️ Digite os sintomas/doenças aqui (ex: febre, tosse persistente, diabetes): covid


Analisando a receita médica manuscrita, considerando a especialidade do médico (Clínico Geral) e os sintomas do paciente (COVID), identifiquei os seguintes medicamentos:

**Item 1:**

*   **Medicamento:** Azitromicina. Embora a grafia inicial "Azi tuaricillia" seja ilegível, a continuação "500mg" e o contexto de tratamento para sintomas de COVID-19 tornam Azitromicina 500mg a identificação mais provável. Azitromicina é um antibiótico macrolídeo frequentemente usado para tratar infecções respiratórias, incluindo pneumonia e bronquite, que podem ser complicações da COVID-19. Alguns protocolos de tratamento para COVID-19 em fases iniciais incluíam Azitromicina.
*   **Posologia:** 500mg, via oral, 1 comprimido por dia. A receita indica "500mg - 5cp", que provavelmente se refere a comprimidos (cp). A linha abaixo parece indicar a frequência. A leitura mais provável é "1 cp por dia".
*   **Duração do Tratamento:** 5 dias. A receita claramente indica "5 dias". Azitromicina para infecções respiratórias é frequentemente prescrita por 3 a 5 dias.
*   **Grau de Confiança:**
    *   Medicamento: ALTO (considerando o contexto médico e a dosagem).
    *   Posologia: MÉDIO (a interpretação de "1 cp por dia" é a mais provável pela caligrafia, mas há uma pequena incerteza).
    *   Duração: ALTO.
*   **Observações/Incertezas:** A grafia inicial do nome do medicamento é bastante difícil de ler. A interpretação de "1 cp por dia" na posologia é baseada na forma como as palavras parecem escritas.

**Item 2:**

*   **Medicamento:** Paracetamol. A grafia "Saurocetamol" é uma provável deturpação da escrita para Paracetamol. A dosagem "500mg" e o uso comum de Paracetamol para febre e dor associados à COVID-19 reforçam essa identificação.
*   **Posologia:** 500mg, via oral, a cada 6 horas se necessário ("6/6h S/N"). A receita indica "500mg - 2cp", que pode se referir a 2 comprimidos de 500mg cada ou a embalagem/quantidade prescrita. No entanto, considerando a posologia usual do Paracetamol para adultos (geralmente 500mg a 1000mg a cada 4-6 horas), e a indicação "6/6h S/N", é mais provável que a dose seja de 500mg a cada 6 horas, se necessário. O "2cp" pode indicar a quantidade total de comprimidos prescritos (por exemplo, 2 caixas). Dada a incerteza, considero a dose como 500mg por administração.
*   **Duração do Tratamento:** Não especificado diretamente na linha do item 2, mas a posologia "S/N" (se necessário) implica que a duração é enquanto os sintomas (febre/dor) persistirem.
*   **Grau de Confiança:**
    *   Medicamento: ALTO (muito provável Paracetamol dado o contexto e dosagem).
    *   Posologia: MÉDIO (incerteza se "2cp" se refere à dose por tomada ou quantidade total, mas a interpretação de 500mg a cada 6h S/N é a mais consistente com a prática e os resultados da pesquisa).
    *   Duração: BAIXO (não explicitamente mencionada, inferida pela posologia S/N).
*   **Observações/Incertezas:** A grafia do nome do medicamento é incomum, parecendo "Saurocetamol". A interpretação de "2cp" na posologia gera incerteza sobre a dose exata por tomada, embora 500mg seja a mais provável no contexto de "500mg" antes do traço.

É importante ressaltar que, como Farmacêutico, esta análise é baseada na interpretação da caligrafia e no conhecimento farmacológico contextualizado com as informações fornecidas (Clínico Geral, sintomas de COVID). Em caso de qualquer dúvida na dispensação, a receita deve ser confirmada com o prescritor.

Agora vou procurar por opções para você adquirir os medicamento. Aguarde um instante

Com base na análise da receita e nas buscas online, seguem as opções de compra encontradas para os medicamentos identificados:

*   Opção 1:
*   **Medicamento:** Azitromicina 500mg
*   **Local:** Drogaria Araujo
*   **Preço:** R$ 9,59 (Genérico Prati com 5 Comprimidos)
*   **URL:** [https://www.drogariaaraujo.com.br/azitromicina-500mg-prati-generico-com-5-comprimidos-revestidos/p](https://www.drogariaaraujo.com.br/azitromicina-500mg-prati-generico-com-5-comprimidos-revestidos/p)

*   Opção 2:
*   **Medicamento:** Azitromicina 500mg
*   **Local:** Drogaria São Paulo
*   **Preço:** R$ 19,49 (Genérico Sandoz do Brasil 5 Comprimidos revestidos)
*   **URL:** [https://www.drogariasaopaulo.com.br/azitromicina-500mg-generico-sandoz-do-brasil-5-comprimidos-revestidos/p](https://www.drogariasaopaulo.com.br/azitromicina-500mg-generico-sandoz-do-brasil-5-comprimidos-revestidos/p)

*   Opção 3:
*   **Medicamento:** Azitromicina 500mg
*   **Local:** Droga Raia
*   **Preço:** A partir de R$ 17,69 (Azitromicina Di-Hidratada EMS 500mg, caixa com 2 comprimidos), Preço não encontrado para 5 comprimidos no snippet. Preço para 5 comprimidos Prati-Donaduzzi não especificado no snippet.
*   **URL:** [https://www.drogaraia.com.br/medicamentos/antibioticos/azitromicina.html](https://www.drogaraia.com.br/medicamentos/antibioticos/azitromicina.html)

---

*   Opção 1:
*   **Medicamento:** Paracetamol 500mg
*   **Local:** Drogaria Rosário
*   **Preço:** R$ 17,01 (Genérico Prati Donaduzzi 20 Comprimidos)
*   **URL:** [https://www.drogaraiarosario.com.br/paracetamol-500mg-20-comprimidos-generico-prati-donaduzzi/p](https://www.drogaraiarosario.com.br/paracetamol-500mg-20-comprimidos-generico-prati-donaduzzi/p)

*   Opção 2:
*   **Medicamento:** Paracetamol 500mg
*   **Local:** CallFarma
*   **Preço:** R$ 14,49 (Prati Donaduzzi Genérico 20 comprimidos)
*   **URL:** [https://www.callfarma.com.br/paracetamol-500mg-20-comprimidos-prati-donaduzzi-generico/p](https://www.callfarma.com.br/paracetamol-500mg-20-comprimidos-prati-donaduzzi-generico/p)

*   Opção 3:
*   **Medicamento:** Paracetamol 500mg
*   **Local:** Drogaria Araujo
*   **Preço:** R$ 16,59 (Prati Genérico com 20 Comprimidos)
*   **URL:** [https://www.drogariaaraujo.com.br/paracetamol-500mg-prati-generico-com-20-comprimidos/p](https://www.drogariaaraujo.com.br/paracetamol-500mg-prati-generico-com-20-comprimidos/p)


### Apoie o Remédio Certo! 🗳️✨

Gostou do **Remédio Certo**? 💊 Ajude essa ideia a decolar! 🚀
Vote no projeto na **Imersão AI Alura + Google**, lá no Discord.
Seu voto faz toda a diferença! 🙏 #RemedioCerto #ImersaoIA
